In [22]:
## Import Required Libraries

import neo4j
import csv
import math
import numpy as np
import pandas as pd
import psycopg2


In [23]:
## Create driver object and session

driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))
session = driver.session(database="neo4j")

In [24]:
## Function to delete any existing graphs

def my_neo4j_wipe_out_database():    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [25]:
## Function to run a query and return the results in a pandas dataframe

def my_neo4j_run_query_pandas(query, **kwargs):    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [26]:
## Function to print the number of nodes and relationships    
    
def my_neo4j_number_nodes_relationships():
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")

In [27]:
## Function to create a node with label Station    
    
def my_neo4j_create_node(station_name):    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)

In [28]:
## Function to create a relationship one way between two stations with travel time as weight

def my_neo4j_create_relationship_one_way(from_station, to_station, weight):    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)

In [29]:
## Function to create relationships two way between two stations with a weight

def my_neo4j_create_relationship_two_way(from_station, to_station, weight):    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)

In [30]:
## Create postgres cursor

connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

cursor = connection.cursor()

In [31]:
## Wipe out any existing graphs

my_neo4j_wipe_out_database()

In [32]:
## Verify

my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 0
  Relationships: 0
-------------------------


In [33]:
## Create the departure and arrival nodes in the graph. Total 50 stations.

connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

print(rows)

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node('depart ' + station)
    my_neo4j_create_node('arrive ' + station)

[('12th Street',), ('16th Street Mission',), ('19th Street',), ('24th Street Mission',), ('Antioch',), ('Ashby',), ('Balboa Park',), ('Bay Fair',), ('Berryessa',), ('Castro Valley',), ('Civic Center',), ('Coliseum',), ('Colma',), ('Concord',), ('Daly City',), ('Downtown Berkeley',), ('Dublin',), ('El Cerrito del Norte',), ('El Cerrito Plaza',), ('Embarcadero',), ('Fremont',), ('Fruitvale',), ('Glen Park',), ('Hayward',), ('Lafayette',), ('Lake Merritt',), ('MacArthur',), ('Millbrae',), ('Milpitas',), ('Montgomery Street',), ('North Berkeley',), ('North Concord',), ('OAK',), ('Orinda',), ('Pittsburg',), ('Pittsburg Center',), ('Pleasant Hill',), ('Powell Street',), ('Richmond',), ('Rockridge',), ('San Bruno',), ('San Leandro',), ('SFO',), ('South Hayward',), ('South San Francisco',), ('Union City',), ('Walnut Creek',), ('Warm Springs',), ('West Dublin',), ('West Oakland',)]


In [34]:
## Verify

my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 100
  Relationships: 0
-------------------------


In [35]:
## Create line nodes, create relationships between the line nodes and the departure and arrival nodes with weight 0

connection.rollback()

query = """

SELECT station, line FROM lines ORDER BY station, line;

"""
cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

print(rows)

for row in rows:
    line_station_name = row[1]+' '+row[0]
    depart_station_name = 'depart ' + row[0]
    arrive_station_name = 'arrive ' + row[0]
    my_neo4j_create_node(line_station_name)
    my_neo4j_create_relationship_one_way(depart_station_name,line_station_name,0)
    my_neo4j_create_relationship_one_way(line_station_name,arrive_station_name,0)

[('12th Street', 'orange'), ('12th Street', 'red'), ('12th Street', 'yellow'), ('16th Street Mission', 'blue'), ('16th Street Mission', 'green'), ('16th Street Mission', 'red'), ('16th Street Mission', 'yellow'), ('19th Street', 'orange'), ('19th Street', 'red'), ('19th Street', 'yellow'), ('24th Street Mission', 'blue'), ('24th Street Mission', 'green'), ('24th Street Mission', 'red'), ('24th Street Mission', 'yellow'), ('Antioch', 'yellow'), ('Ashby', 'orange'), ('Ashby', 'red'), ('Balboa Park', 'blue'), ('Balboa Park', 'green'), ('Balboa Park', 'red'), ('Balboa Park', 'yellow'), ('Bay Fair', 'blue'), ('Bay Fair', 'green'), ('Bay Fair', 'orange'), ('Berryessa', 'green'), ('Berryessa', 'orange'), ('Castro Valley', 'blue'), ('Civic Center', 'blue'), ('Civic Center', 'green'), ('Civic Center', 'red'), ('Civic Center', 'yellow'), ('Coliseum', 'blue'), ('Coliseum', 'gray'), ('Coliseum', 'green'), ('Coliseum', 'orange'), ('Colma', 'red'), ('Colma', 'yellow'), ('Concord', 'yellow'), ('Daly 

In [36]:
## Verify

my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 228
-------------------------


In [37]:
## Create a relationship for each possible transfer with the transfer time as the weight

connection.rollback()

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines a
     join lines b
       on a.station = b.station and a.line <> b.line 
     join stations s
       on a.station = s.station
order by 1, 2, 3

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    from_ln_statn_nm = row[1]+' '+row[0]
    to_ln_statn_nm = row[2]+' '+row[0]
    my_neo4j_create_relationship_one_way(from_ln_statn_nm, to_ln_statn_nm, int(row[3]))

In [38]:
## Verify

my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 436
-------------------------


In [39]:
## Create a relationship between each station and its adjoining stations both ways

connection.rollback()

query = """
SELECT
    l1.line as line,
    l1.station as from_station,
    l2.station as to_station,
    tt.travel_time as travel_time_in_seconds
FROM lines l1
INNER JOIN lines l2
ON l1.line = l2.line
INNER JOIN travel_times tt
ON (l1.station = tt.station_1 AND l2.station = tt.station_2) OR (l2.station = tt.station_1 AND l1.station = tt.station_2)
WHERE l1.station != l2.station AND l1.sequence=(l2.sequence-1)
ORDER BY l1.line, l1.station, l2.station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

print(rows)

for row in rows:
    from_ln_statn_nm = row[0]+' '+row[1]
    to_ln_statn_nm = row[0]+' '+row[2]
    my_neo4j_create_relationship_two_way(from_ln_statn_nm, to_ln_statn_nm, int(row[3]))

[('blue', '16th Street Mission', '24th Street Mission', Decimal('120')), ('blue', '24th Street Mission', 'Glen Park', Decimal('180')), ('blue', 'Balboa Park', 'Daly City', Decimal('240')), ('blue', 'Bay Fair', 'San Leandro', Decimal('240')), ('blue', 'Castro Valley', 'Bay Fair', Decimal('240')), ('blue', 'Civic Center', '16th Street Mission', Decimal('180')), ('blue', 'Coliseum', 'Fruitvale', Decimal('240')), ('blue', 'Dublin', 'West Dublin', Decimal('180')), ('blue', 'Embarcadero', 'Montgomery Street', Decimal('60')), ('blue', 'Fruitvale', 'Lake Merritt', Decimal('300')), ('blue', 'Glen Park', 'Balboa Park', Decimal('120')), ('blue', 'Lake Merritt', 'West Oakland', Decimal('360')), ('blue', 'Montgomery Street', 'Powell Street', Decimal('120')), ('blue', 'Powell Street', 'Civic Center', Decimal('60')), ('blue', 'San Leandro', 'Coliseum', Decimal('240')), ('blue', 'West Dublin', 'Castro Valley', Decimal('600')), ('blue', 'West Oakland', 'Embarcadero', Decimal('420')), ('gray', 'OAK', 'C

In [40]:
## Verify

my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 652
-------------------------
